<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/Esik%C3%A4sittely_ja_talennus_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Tuonnit ja Asetukset (Päivitetty Esikäsittely v3)

import pandas as pd
import numpy as np
import requests
import io
import os
import re # Tuodaan regex pilvisyyden käsittelyyn
import traceback # Virheiden jäljitykseen

# --- Tiedostojen URL-osoitteet GitHubissa ---
WEATHER_CSV_URL = 'https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kaisaniemi_%201.4.2024%20-%201.4.2025_bb4e130b-02c8-489d-8a07-41e8b216a5b5.csv'
AIRQUALITY_CSV_URL = 'https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kallio%202_%201.4.2024%20-%201.4.2025_8bbf9745-4df3-4828-ba50-78e3ad8410b4.csv'

# --- Tallennuspolku ja tiedostonimi ---
output_dir = 'data/processed'
os.makedirs(output_dir, exist_ok=True)
OUTPUT_FILENAME = os.path.join(output_dir, 'processed_Helsinki_O3_Weather_Cloudiness_2024_2025_v3.parquet') # Lisätty v3 nimeen

# --- Valitut ja uudelleennimetyt sarakkeet ---
# Avaimet ovat TÄSMÄLLEEN kuten CSV-tiedostossa, arvot ovat halutut lopulliset nimet

WEATHER_COLS_TO_KEEP = {
    # Aikaleima luodaan erikseen
    'Ilman lämpötila keskiarvo [°C]': 'Lämpötilan keskiarvo [°C]',
    'Keskituulen nopeus keskiarvo [m/s]': 'Keskituulen nopeus [m/s]',
    'Tuulen suunta keskiarvo [°]': 'Tuulen suunnan keskiarvo [°]',
    'Ilmanpaine merenpinnan tasolla keskiarvo [hPa]': 'Ilmanpaineen keskiarvo [hPa]',
    'Pilvisyys [1/8]': 'Pilvisyys [okta]' # Tämä parsitaan myöhemmin numeroksi
}

AIRQUALITY_COLS_TO_KEEP = {
    # Aikaleima luodaan erikseen
    'Otsoni [µg/m3]': 'Otsoni [µg/m³]' # Huom: muutettu yksikkö haluttuun muotoon
}

# Lopulliset sarakkeet yhdistetyssä DataFramessa (tähän tulee vain numeriset sarakkeet)
FINAL_COLUMNS = [
    'Otsoni [µg/m³]',
    'Lämpötilan keskiarvo [°C]',
    'Keskituulen nopeus [m/s]',
    'Ilmanpaineen keskiarvo [hPa]',
    'Tuulen suunnan keskiarvo [°]',
    'Pilvisyys [okta]' # Tämä on nyt numeerinen okta-arvo
]

print("Osa 1: Tuonnit ja Asetukset - OK")

In [ ]:
# @title 2. Datan Lataus ja Esikäsittely Funktiot (Päivitetty Esikäsittely v3)

def download_and_read_csv(url):
    """Lataa CSV-datan annetusta URL:sta ja lukee sen DataFrameen."""
    print(f"Ladataan dataa osoitteesta: {url[:60]}...")
    try:
        response = requests.get(url)
        response.raise_for_status()
        # skipinitialspace=True auttaa, jos sarakenimissä on alussa välilyöntejä
        df = pd.read_csv(io.StringIO(response.text), sep=',', decimal='.', skipinitialspace=True)
        print(f"Ladattu {len(df)} riviä.")
        # Korvataan mahdolliset '-' merkit NaN-arvoilla numeerisissa sarakkeissa jo tässä vaiheessa
        # Tämä auttaa myöhemmässä käsittelyssä
        df = df.replace('-', np.nan)
        return df
    except requests.exceptions.RequestException as e:
        print(f"Virhe datan latauksessa URL:sta {url}: {e}")
        return None
    except Exception as e:
        print(f"Odottamaton virhe ladattaessa ja lukiessa dataa URL:sta {url}: {e}")
        return None

def process_datetime_local(df):
    """Yhdistää aika-sarakkeet ja luo aikaleimaindeksin olettaen paikallista aikaa."""
    print("Käsitellään aikaleimoja (oletus: Paikallinen aika)...")
    try:
        required_time_cols = ['Vuosi', 'Kuukausi', 'Päivä', 'Aika [Paikallinen aika]']
        if not all(col in df.columns for col in required_time_cols):
            missing = [col for col in required_time_cols if col not in df.columns]
            print(f"VIRHE: Puuttuvat aikasarakkeet aikaleiman luontiin: {missing}")
            return None

        # Puhdistetaan Aika-sarake (poistetaan mahdolliset epävalidit merkinnät)
        # Oletetaan muoto hh:mm
        df['Time_Str_Clean'] = df['Aika [Paikallinen aika]'].astype(str).str.extract(r'(\d{1,2}:\d{2})')[0]
        # Täytetään puuttuvat ajat esim. 00:00, jotta to_datetime ei kaadu, mutta merkitään ne
        invalid_times = df['Time_Str_Clean'].isnull()
        if invalid_times.any():
            print(f"VAROITUS: Löydettiin {invalid_times.sum()} epävalidia aika-merkintää, asetetaan 00:00.")
            df.loc[invalid_times, 'Time_Str_Clean'] = '00:00'


        # Yhdistetään sarakkeet merkkijonoksi (käytetään zfill varmistamaan kk ja pv pituus)
        datetime_str = df['Vuosi'].astype(str) + '-' + \
                       df['Kuukausi'].astype(str).str.zfill(2) + '-' + \
                       df['Päivä'].astype(str).str.zfill(2) + ' ' + \
                       df['Time_Str_Clean']

        # Muunnetaan datetime-objekteiksi, virheet muutetaan NaT (Not a Time)
        df['Aikaleima'] = pd.to_datetime(datetime_str, format='%Y-%m-%d %H:%M', errors='coerce')

        # Tarkistetaan NaT-arvot
        nat_values = df['Aikaleima'].isnull()
        if nat_values.any():
            print(f"VAROITUS: Löydettiin {nat_values.sum()} virheellistä päivämäärä/aika-yhdistelmää (muutettu NaT).")
            # Voitaisiin poistaa nämä rivit: df = df.dropna(subset=['Aikaleima'])
            # Tässä annetaan niiden olla ja käsitellään myöhemmin

        # Asetetaan aikavyöhykkeeksi Europe/Helsinki
        try:
            df['Aikaleima'] = df['Aikaleima'].dt.tz_localize('Europe/Helsinki', ambiguous='infer', nonexistent='shift_forward')
            print("Aikaleimoille asetettu aikavyöhyke Europe/Helsinki.")
        except Exception as e_tz:
            print(f"VAROITUS: Aikavyöhykkeen asetus Europe/Helsinki epäonnistui: {e_tz}")
            # Jos lokalisointi epäonnistuu, indeksi jää naiiviksi

        # Poistetaan väliaikainen aikasarae
        df.drop(columns=['Time_Str_Clean'], inplace=True, errors='ignore')

        # Asetetaan aikaleima indeksiksi ja poistetaan NaT-rivit
        df.set_index('Aikaleima', inplace=True)
        rows_before = len(df)
        df.dropna(axis=0, how='all', inplace=True) # Poista rivit jotka ovat kokonaan NaN
        df.dropna(inplace=True) # Poista rivit joilla on yksikin NaN indeksissä (NaT)
        rows_after = len(df)
        if rows_before > rows_after:
            print(f"Poistettu {rows_before - rows_after} riviä virheellisten aikaleimojen (NaT) vuoksi.")

        df.sort_index(inplace=True)
        print("Aikaleimaindeksi luotu ja asetettu.")
        return df

    except Exception as e:
        print(f"VIRHE aikaleimojen käsittelyssä: {e}")
        traceback.print_exc()
        return None

def parse_cloudiness(cloud_text):
    """Poimii numeerisen okta-arvon tekstimuotoisesta pilvisyydestä."""
    if pd.isna(cloud_text):
        return np.nan
    if isinstance(cloud_text, (int, float, np.number)): # Jos on jo numero
        return float(cloud_text)

    # Yritä löytää numero sulkujen sisältä (esim. "(7/8)")
    match = re.search(r'\((\d+)/8\)', str(cloud_text))
    if match:
        return float(match.group(1)) # Palauta numero (0-8)

    # Lisätään tarkistus yleisille termeille (jos sulkuja ei ole)
    text_lower = str(cloud_text).lower()
    if "selkeää" in text_lower: return 0.0
    if "puolipilvistä" in text_lower: return 4.0 # Arvaus
    if "melko selkeää" in text_lower: return 2.0 # Arvaus
    if "melko pilvistä" in text_lower: return 6.0 # Arvaus
    if "pilvistä" in text_lower: return 8.0

    # Jos mikään ei täsmää, palauta NaN
    # print(f"Ei voitu parsia pilvisyyttä: '{cloud_text}'") # Debug-tuloste tarvittaessa
    return np.nan

def select_and_rename(df, columns_dict):
    """Valitsee ja uudelleennimeää sarakkeet annetun sanakirjan mukaisesti."""
    if df is None: return None
    cols_to_select = [col for col in columns_dict.keys() if col in df.columns]
    if not cols_to_select:
        print(f"VAROITUS: Yhtään määriteltyä saraketta ({list(columns_dict.keys())}) ei löytynyt DataFramesta.")
        return pd.DataFrame(index=df.index) # Palauta tyhjä df samalla indeksillä

    rename_map = {k: columns_dict[k] for k in cols_to_select}

    try:
        df_selected = df[cols_to_select].copy()
        df_renamed = df_selected.rename(columns=rename_map)
        print(f"Valittu ja uudelleennimetty sarakkeet: {df_renamed.columns.tolist()}")
        return df_renamed
    except Exception as e:
        print(f"VIRHE sarakkeiden valinnassa/uudelleennimeämisessä: {e}")
        return None

print("Osa 2: Funktiot datan käsittelyyn - OK")

In [ ]:
# @title 3. Datan Lataus, Yhdistäminen ja Puhdistus (Päivitetty Esikäsittely v3)

print("--- Aloitetaan datan lataus ja käsittely ---")
df_final = None # Alustetaan Noneksi

try: # Lisätään try-except koko lohkon ympärille

    # 1. Lataa säädata
    df_weather_raw = download_and_read_csv(WEATHER_CSV_URL)

    # 2. Lataa ilmanlaatudata
    df_airquality_raw = download_and_read_csv(AIRQUALITY_CSV_URL)

    # Tarkistetaan, että molemmat lataukset onnistuivat
    if df_weather_raw is not None and df_airquality_raw is not None:

        # 3. Käsittele aikaleimat molemmille (käyttäen päivitettyä funktiota)
        df_weather_time = process_datetime_local(df_weather_raw)
        df_airquality_time = process_datetime_local(df_airquality_raw)

        # Tarkistetaan, että aikaleimojen käsittely onnistui
        if df_weather_time is not None and df_airquality_time is not None:

            # 4. *** UUSI VAIHE: Käsittele Pilvisyys ***
            cloud_col_raw = 'Pilvisyys [1/8]'
            cloud_col_target = 'Pilvisyys [okta]'
            if cloud_col_raw in df_weather_time.columns:
                print(f"\nKäsitellään sarake '{cloud_col_raw}'...")
                # Muutetaan ensin kaikki numeroksi, jos mahdollista
                df_weather_time[cloud_col_raw] = pd.to_numeric(df_weather_time[cloud_col_raw], errors='ignore')
                # Käytä parse_cloudiness funktiota
                df_weather_time[cloud_col_target] = df_weather_time[cloud_col_raw].apply(parse_cloudiness)
                # Muunnetaan lopuksi numeeriseksi tyypiksi (float)
                df_weather_time[cloud_col_target] = pd.to_numeric(df_weather_time[cloud_col_target], errors='coerce')
                print(f"Pilvisyys parsittu numeeriseen muotoon (sarake '{cloud_col_target}').")
                # Tarkistetaan montako NaN arvoa jäi
                nan_count = df_weather_time[cloud_col_target].isnull().sum()
                if nan_count > 0:
                    print(f"VAROITUS: Pilvisyys-sarakkeessa {nan_count} NaN arvoa parsimisen jälkeen.")
                # Poistetaan alkuperäinen tekstimuotoinen sarake (jos se on eri nimi)
                if cloud_col_raw != cloud_col_target:
                     df_weather_time.drop(columns=[cloud_col_raw], inplace=True, errors='ignore')
            else:
                print(f"VAROITUS: Pilvisyyssaraketta '{cloud_col_raw}' ei löytynyt säädatasta.")
                # Lisätään tyhjä sarake, jotta merge ei kaadu, jos se on FINAL_COLUMNSissa
                if cloud_col_target in FINAL_COLUMNS:
                    df_weather_time[cloud_col_target] = np.nan


            # 5. Valitse ja nimeä sarakkeet uudelleen
            # Huom: Nyt WEATHER_COLS_TO_KEEP avaimena on tekstimuotoinen nimi,
            # mutta DataFramessa on jo numeerinen 'Pilvisyys [okta]'. Korjataan tämä.
            # Poistetaan vanha pilvisyysavain dictistä ja käytetään vain jo luotua saraketta.
            weather_cols_final_map = {k: v for k, v in WEATHER_COLS_TO_KEEP.items() if v != 'Pilvisyys [okta]'}
            # Valitaan ja nimetään muut sääsarakkeet
            df_weather_renamed_partial = select_and_rename(df_weather_time, weather_cols_final_map)

            # Lisätään numeerinen pilvisyyssarake takaisin, jos se luotiin
            if df_weather_renamed_partial is not None and cloud_col_target in df_weather_time.columns:
                 df_weather = pd.concat([df_weather_renamed_partial, df_weather_time[[cloud_col_target]]], axis=1)
                 # Poista rivit joissa pilvisyys on NaN (jos tuli parsimisvirheitä)
                 rows_before_na = len(df_weather)
                 df_weather.dropna(subset=[cloud_col_target], inplace=True)
                 if len(df_weather) < rows_before_na:
                      print(f"Poistettu {rows_before_na - len(df_weather)} riviä NaN-pilvisyyden vuoksi.")
            elif df_weather_renamed_partial is not None:
                 df_weather = df_weather_renamed_partial # Jos pilvisyyttä ei ollut/luotu
            else:
                 df_weather = None # Jos select_and_rename epäonnistui


            # Valitaan ja nimetään ilmanlaatusarakkeet
            df_airquality = select_and_rename(df_airquality_time, AIRQUALITY_COLS_TO_KEEP)

            # Tarkistetaan, että kaikki onnistui tähän asti
            if df_weather is not None and df_airquality is not None:

                # 6. Yhdistä DataFramet aikaleimaindeksin perusteella
                print("\nYhdistetään sää- ja ilmanlaatudata...")
                df_merged = pd.merge(df_weather, df_airquality, left_index=True, right_index=True, how='outer')
                print(f"Yhdistetty DataFrame, muoto: {df_merged.shape}")
                print(f"Yhdistämisen jälkeen puuttuvat arvot:\n{df_merged.isnull().sum()}")

                # 7. Käsittele puuttuvat arvot yhdistämisen jälkeen
                print("\nTäytetään puuttuvat arvot yhdistämisen jälkeen (ffill + bfill)...")
                df_merged.ffill(inplace=True)
                df_merged.bfill(inplace=True)

                final_nulls = df_merged.isnull().sum()
                print(f"\nPuuttuvat arvot lopullisessa datassa:\n{final_nulls}")
                if final_nulls.sum() > 0:
                     print("VAROITUS: Data sisältää edelleen puuttuvia arvoja täytön jälkeen! Poistetaan rivit.")
                     df_merged.dropna(inplace=True)
                     print(f"Muoto NaN-rivien poiston jälkeen: {df_merged.shape}")

                # 8. Varmista sarakkeiden tyypit (kaikki numeerisia)
                print("\nVarmistetaan sarakkeiden datatyypit...")
                for col in FINAL_COLUMNS:
                    if col in df_merged.columns:
                         try:
                              df_merged[col] = pd.to_numeric(df_merged[col])
                         except ValueError:
                              print(f"VAROITUS: Saraketta '{col}' ei voitu muuttaa numeeriseksi.")
                              df_merged[col] = np.nan # Aseta NaN jos ei onnistu
                    else:
                         print(f"VAROITUS: Lopullinen sarake '{col}' puuttuu yhdistetystä datasta.")
                         # Lisätään sarake ja täytetään NaN (tai 0)
                         df_merged[col] = np.nan
                # Poistetaan rivit, joissa on NaN arvoja tyyppimuunnoksen jälkeen
                rows_before_final_na = len(df_merged)
                df_merged.dropna(subset=FINAL_COLUMNS, inplace=True)
                if len(df_merged) < rows_before_final_na:
                     print(f"Poistettu {rows_before_final_na - len(df_merged)} riviä lopullisten NaN-arvojen vuoksi.")


                # 9. Valitse ja järjestä lopulliset sarakkeet
                missing_final_cols = [col for col in FINAL_COLUMNS if col not in df_merged.columns]
                if missing_final_cols:
                    print(f"VIRHE: Seuraavat lopulliset sarakkeet puuttuvat: {missing_final_cols}")
                    raise ValueError("Lopullisia sarakkeita puuttuu.")

                df_final = df_merged[FINAL_COLUMNS].copy()

                # 10. Tallenna käsitelty data Parquet-tiedostoon
                print(f"\nTallennetaan käsitelty data tiedostoon: {OUTPUT_FILENAME}")
                df_final.to_parquet(OUTPUT_FILENAME)
                print("Tallennus onnistui!")
                print(f"\nLopullisen datan muoto: {df_final.shape}")
                if not df_final.empty:
                    print(f"Lopullisen datan aikaväli: {df_final.index.min()} - {df_final.index.max()}")
                    print("\nEnsimmäiset 5 riviä:")
                    print(df_final.head())
                    print("\nViimeiset 5 riviä:")
                    print(df_final.tail())
                    print("\nDatan perustiedot (info):")
                    df_final.info()
                    print("\nDatan tilastolliset tunnusluvut (describe):")
                    print(df_final.describe())
                else:
                    print("VAROITUS: Lopullinen DataFrame on tyhjä!")

            else: print("VIRHE: Sää- tai ilmanlaatudatan käsittely epäonnistui ennen yhdistämistä.")
        else: print("VIRHE: Aikaleimojen käsittely epäonnistui toiselle tai molemmille DataFramelle.")
    else: print("VIRHE: Datan lataus epäonnistui toiselle tai molemmille tiedostolle.")

except Exception as e:
     print(f"\n---> ODOTTAMATON VIRHE DATAN KÄSITTELYSSÄ (Osa 3) <---")
     print(f"Virhetyyppi: {type(e).__name__}")
     print(f"Virheilmoitus: {e}")
     traceback.print_exc()
     print("----------------------------------------------")


if df_final is not None and not df_final.empty:
     print("\nOsa 3: Datan käsittely ja tallennus - VALMIS")
else:
     print("\nOsa 3: Datan käsittely ja tallennus - EPÄONNISTUNUT / Data tyhjä")